# Image Classification Using CNN
By : Dhia Chelly&Emna Abd Mouleh

________

we'll build a CNN using Keras to use it classifying pictures in 3 different categories


first to import libraries


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set(style="whitegrid")
import os
import glob as gb
import cv2
import tensorflow as tf
import keras

extraire les dossiers des images



In [ ]:
!unzip /DATABASE.zip   

now to define the path

In [ ]:
#now to define the path
trainpath ='/DATABASE/TRAIN/'
testpath ='/DATABASE/VAL/'
predpath= '/DATABASE/TEST/'

# Open Folders

now let's first check the Train folder to have a look to its content

In [ ]:
for folder in  os.listdir(trainpath) : 
    files = gb.glob(pathname= str( trainpath  + folder + '/*.jpg'))
    print(f'For training data , found {len(files)} in folder {folder}')

ok , how about the test folder

In [ ]:
for folder in  os.listdir(testpath) : 
    files = gb.glob(pathname= str(testpath+ folder + '/*.jpg'))
    print(f'For testing data , found {len(files)} in folder {folder}')

ok , how about the test folder

In [ ]:
for folder in  os.listdir(predpath) : 
  files = gb.glob(pathname= str(predpath+ folder + '/*.jpg'))
  print(f'For testing data , found {len(files)} in folder {folder}')

_____

# Checking Images

now we need to heck the images sizes , to know ow they looks like

since we have 5 categories , we first need to create a dictionary with their names & indices , also create a function to get the code back

In [ ]:
code = {'dandelion':0,'rose':1,'sunflower':2}

def getcode(n) : 
    for x , y in code.items() : 
        if n == y : 
            return x    

now how about the images sizes in train folder

In [ ]:
size = []
for folder in  os.listdir(trainpath) : 
    files = gb.glob(pathname= str( trainpath + folder + '/*.jpg'))
    for file in files: 
        image = plt.imread(file)
        size.append(image.shape)
pd.Series(size).value_counts()

______

ok , min size : 240,215,3 , how about test images ? 

In [ ]:
size = []
for folder in  os.listdir(testpath) : 
    files = gb.glob(pathname= str( testpath + folder + '/*.jpg'))
    for file in files: 
        image = plt.imread(file)
        size.append(image.shape)
pd.Series(size).value_counts()

 now to prediction images 

In [ ]:
size = []
for folder in  os.listdir(predpath) : 
    files = gb.glob(pathname= str( predpath + folder + '/*.jpg'))
    for file in files: 
        image = plt.imread(file)
        size.append(image.shape)
pd.Series(size).value_counts()

# Reading Images

now it's time to read all images & convert it into arrays

first we'll create a variable s , which refer to size , so we can change it easily 

let's use now size = 200 , so it will be suitable amount to contain accuracy without losing so much time in training

In [ ]:
s = 175

now to read all pictues in 5 categories in training folder, ans use OpenCV to resize it , and not to forget to assign the y value , from the predefined function 

In [ ]:
X_train = []
y_train = []
for folder in  os.listdir(trainpath) : 
    files = gb.glob(pathname= str( trainpath+ folder + '/*.jpg'))
    for file in files: 
        image = cv2.imread(file)
        image_array = cv2.resize(image , (s,s))
        X_train.append(list(image_array))
        y_train.append(code[folder])

great , now how many items in X_train 

In [ ]:
print(f'we have {len(X_train)} items in X_train')

also we have have a look to random pictures in X_train , and to adjust their title using the y value

great , now to repeat same steps exactly in test data

In [ ]:
X_test = []
y_test = []
for folder in  os.listdir(testpath) : 
    files = gb.glob(pathname= str(testpath + folder + '/*.jpg'))
    for file in files: 
        image = cv2.imread(file)
        image_array = cv2.resize(image , (s,s))
        X_test.append(list(image_array))
        y_test.append(code[folder])
        

In [ ]:
print(f'we have {len(X_test)} items in X_test')

also with Prediction data , without having title ofcourse

In [ ]:
X_pred = []
for folder in  os.listdir(testpath) : 
   files = gb.glob(pathname= str(predpath+ folder + '/*.jpg'))
   for file in files: 
    image = cv2.imread(file)
    image_array = cv2.resize(image , (s,s))
    X_pred.append(list(image_array))       

In [ ]:
print(f'we have {len(X_pred)} items in X_pred')

ok ,  we can feel comfort in using all pictures in our model , after resizing it in a specific amount

________

# Building The Model 

now we need to build the model to train our data

first to convert the data into arrays using numpy

In [ ]:
X_train = np.array(X_train)
X_test = np.array(X_test)
X_pred_array = np.array(X_pred)
y_train = np.array(y_train)
y_test = np.array(y_test)

print(f'X_train shape  is {X_train.shape}')
print(f'X_test shape  is {X_test.shape}')
print(f'X_pred shape  is {X_pred_array.shape}')
print(f'y_train shape  is {y_train.shape}')
print(f'y_test shape  is {y_test.shape}')

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range = 20,  # randomly rotate images in the range (degrees, 0 to 180)
        zoom_range = 0.4, # Randomly zoom image 
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip = True,  # randomly flip images
        vertical_flip=False)  # randomly flip images


datagen.fit(X_train)

now to build the CNN model by Keras , using Conv2D layers , MaxPooling & Denses

In [ ]:
KerasModel = keras.models.Sequential([
        keras.layers.Conv2D(32,(3,3),activation='relu',input_shape=(s,s,3)),
        keras.layers.MaxPool2D(2,2, padding="same"),
        keras.layers.Dropout(rate=0.2) , 
        keras.layers.Conv2D(64,kernel_size=(3,3),activation='relu'), 
        keras.layers.MaxPool2D(2,2, padding="same"),
        keras.layers.Dropout(rate=0.2) , 
        keras.layers.Conv2D(128,kernel_size=(3,3),activation='relu'), 
        keras.layers.MaxPool2D(2,2, padding="same"),
        keras.layers.Dropout(rate=0.2) , 
        keras.layers.Flatten() ,    
        keras.layers.Dense(512,activation='relu') ,           
        keras.layers.Dropout(rate=0.2) ,  
        keras.layers.Dense(128,activation='relu') ,           
        keras.layers.Dropout(rate=0.2) ,            
        keras.layers.Dense(3,activation='softmax') ,    
        ])

now to compile the model , using adam optimizer , & sparse categorical crossentropy loss

In [ ]:
KerasModel.compile(optimizer ='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])

so how the model looks like ? 

In [ ]:
print('Model Details are : ')
print(KerasModel.summary())

now to train the model , lets use 20 epochs now


In [ ]:
epochs = 20
ThisModel = KerasModel.fit(X_train, y_train, epochs=epochs,batch_size=128,verbose=1)

how is the final loss & accuracy


In [ ]:
ModelLoss, ModelAccuracy = KerasModel.evaluate(X_test, y_test)

print('Test Loss is {}'.format(ModelLoss))
print('Test Accuracy is {}'.format(ModelAccuracy ))

** ok** , only **76**% accuracy & can be increased by tuning the hyperparameters


In [ ]:
y_pred = KerasModel.predict(X_test)

print('Prediction Shape is {}'.format(y_pred.shape))

great

now it's time to redict X Predict

In [ ]:
y_result = KerasModel.predict(X_pred_array)

print('Prediction Shape is {}'.format(y_result.shape))

and to show random redicted pictures & its predicting category


In [ ]:
plt.figure(figsize=(20,20))
for  i in range( len(X_pred)) : 
    plt.subplot(6,5,i+1)
    plt.imshow(X_pred[i])    
    plt.axis('off')
    plt.title(getcode(np.argmax(y_result[i])))